In [1]:
from utilities import *
from subprocess import run, PIPE

In [2]:
def print_variables():
    import utilities
    print("Variables: {0}".format(utilities._vars))
    
print_variables()

Variables: ['R', 'pR', 'Theta', 'pTheta', 'phi', 'p_phi', 'theta', 'p_theta', 'psi', 'p_psi']


In [3]:
ps_point = PS_point(15.0, -6.00393, 0.753412, -21.1339, 3.64236, 21.3157, 0.487732, 193.326, 0.979626, 7.99349)

In [4]:
local_path = "/home/artfin/Desktop/repos/generalHamDerivatives/v1.0/trajectory-python-integration/"

def create_wrapper( f, ps_point, var_name ):
    return lambda x: f( ps_point._replace(**{var_name: x}) )
    
h = 1.0e-4
def finite_diff( f, ps_point, var_name, npoints = 5 ):
    wrapper = create_wrapper(f, ps_point, var_name)
    x = getattr(ps_point, var_name)
    if npoints == 3:
        return (wrapper(x + h) - wrapper(x - h)) / (2.0*h)
    elif npoints == 5:
        return (wrapper(x - 2.0*h) - 8.0*wrapper(x - h) + 8.0*wrapper(x + h) - wrapper(x + 2.0*h)) / (12.0 * h) 
    elif npoints == 7:
        return (-wrapper(x - 3.0*h) + 9.0*wrapper(x - 2.0*h) - 45.0*wrapper(x - h) + 45.0*wrapper(x + h) - 9.0*wrapper(x + 2.0*h) + wrapper(x + 3.0*h)) / (60.0 * h) 
    
def num_diff_hamiltonian(f, ps_point, npoints = 5):
    dvars = [ finite_diff(f, ps_point, ps_point._fields[k + 1], npoints) if k % 2 == 0 
             else -finite_diff(f, ps_point, ps_point._fields[k - 1])
             for k in range(len(ps_point._fields))
            ]
    return dPS_point( *dvars )

In [5]:
print(compute_hamiltonian( ps_point ))

0.0035329391187156


In [6]:
analytical_dps_point = compute_hamiltonian_derivatives( ps_point )
numerical_dps_point  = num_diff_hamiltonian(compute_hamiltonian, ps_point)

print('Variable name \t analytical - numerical')
for var_name in dPS_point._fields:
    print('{:>8} {:>20.2e}'.format(
            var_name, 
            getattr(analytical_dps_point, var_name) - getattr(numerical_dps_point, var_name))
          )

Variable name 	 analytical - numerical
      dR             1.24e-13
     dpR            -3.64e-13
  dTheta            -2.99e-13
 dpTheta             6.22e-13
    dphi             1.34e-13
  dp_phi            -1.08e-15
  dtheta            -2.21e-14
dp_theta            -1.00e-13
    dpsi             2.05e-13
  dp_psi            -1.76e-13
